# Load Data

In [1]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_Multi_Raw')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

print(data_test.head())

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_Multi_Raw
   day  month  year  is_holiday    location  temperature
0   10      5  2022       False       Rehau         7.74
1   20      9  2020       False  Holzminden         9.37
2   12      5  2020       False    Grafenau         8.97
3    1      4  2020       False     Parchim        10.05
4   23      8  2022       False     Ansbach         7.32


In [2]:
import numpy as np

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Parameters for multi-item newsvendor problem
prices = np.array([0.3, 0.5, 0.6, 0.5, 0.5, 0.5]) #price data
costs = np.array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06]) #cost data
salvages = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01]) #salvage data
underage_data = prices - costs 
overage_data = costs - salvages 


alpha_data = np.array([             #alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
])


# Preprocessing

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define preprocessing for numeric columns (scale them)
numeric_features = ['temperature']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features (encode them)
categorical_features = ['location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')

# Preprocessing on train data
X_train = preprocessor.fit_transform(data_train)

# Preprocessing on test data
X_test = preprocessor.transform(data_test)


print(X_test)


[[-1.2614810567465224 0.0 0.0 ... 5 2022 False]
 [0.03038460101498815 0.0 0.0 ... 9 2020 False]
 [-0.2866376462884491 0.0 0.0 ... 5 2020 False]
 ...
 [-1.2614810567465224 0.0 0.0 ... 1 2021 False]
 [-1.0316399274515304 0.0 0.0 ... 1 2022 False]
 [0.03038460101498815 0.0 0.0 ... 4 2020 True]]


# ANN

In [4]:
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
from scipy.stats import reciprocal
from sklearn.model_selection import KFold 

In [5]:
def make_nvps_loss(alpha, u, o):

    # transofrm the alpha, u, o to tensors
    u = tf.convert_to_tensor(underage_data, dtype=tf.float32) #underage costs
    o = tf.convert_to_tensor(overage_data, dtype=tf.float32) #overage costs
    alpha = tf.convert_to_tensor(alpha_data, dtype=tf.float32) #substitution matrix

    # define the loss function
    @tf.autograph.experimental.do_not_convert
    def nvps_loss(y_true, y_pred):
        q = tf.maximum(y_pred, 0.)

        # Calculate the demand increase for each product due to substitutions from other products
        demand_increase = tf.matmul( tf.maximum(0.0, y_true - y_pred),alpha)
        # Adjusted demand is the original demand plus the increase due to substitutions
        adjusted_demand = y_true + demand_increase

        profits = tf.matmul(q,tf.transpose(u)) - tf.matmul(tf.maximum(0.0,q - adjusted_demand), tf.transpose(u+o))

        return -tf.math.reduce_mean(profits)
    
    return nvps_loss

In [6]:
def loss_complex(y_true, y_pred):

    # Cast numpy arrays to tensors
    u = tf.convert_to_tensor(underage_data, dtype=tf.float32) #underage costs
    o = tf.convert_to_tensor(overage_data, dtype=tf.float32) #overage costs
    alpha = tf.convert_to_tensor(alpha_data, dtype=tf.float32) #substitution matrix

    # Cast y_true to float32
    y_true = tf.cast(y_true, dtype=tf.float32)

    # Calculate the demand increase for each product due to substitutions from other products
    demand_increase = tf.matmul( tf.maximum(0.0, y_true - y_pred),alpha)

    # Adjusted demand is the original demand plus the increase due to substitutions
    adjusted_demand = y_true + demand_increase

    # Compute the loss with adjusted demand
    loss = -tf.reduce_mean(u * y_pred - (u + o) * tf.maximum(y_pred - adjusted_demand, 0))
    return loss

from keras.utils import get_custom_objects
get_custom_objects().update({'loss_complex': loss_complex})

# Model creation function 
def create_model(n_hidden, n_neurons, learning_rate, activation):
    model = Sequential()
    # Input Layer
    model.add(Dense(n_neurons,input_dim=15, activation=activation))
    # Hidden Layer
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation=activation))
    # Output Layer
    model.add(Dense(6))
    model.compile(loss = loss_complex,
                  optimizer=Adam(learning_rate=learning_rate))
    return model

# Model builder function 
def model_builder(n_hidden, n_neurons, learning_rate, activation, batch_size, epochs):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, 
                          learning_rate=learning_rate, activation=activation, batch_size=batch_size, epochs=epochs)

# Create a baseline model
model_ANN = model_builder(1,30,3e-3,'relu', 32, 20)


# Define the parameter grid
from scipy.stats import reciprocal
params = {
    "n_hidden": range(0, 15),
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10,15, 20, 25, 30],
    "activation": ['relu', 'sigmoid', 'tanh'] #
}

# Optimize the model using RandomizedSearchCV
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distributions=params, cv=KFold(10))

# Fit the model
rnd_search_cv_ANN.fit(X_train, target_train)
print(rnd_search_cv_ANN.best_params_)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\keras

ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py", line 760, in fit
    self._fit(
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py", line 928, in _fit
    self._fit_keras_model(
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py", line 536, in _fit_keras_model
    raise 

In [ ]:
# Predict with best model
best_model_ANN = rnd_search_cv_ANN.best_estimator_
best_model_ANN.fit(X_train, target_train)
target_pred_ANN = best_model_ANN.predict(X_test)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


KerasRegressor(
	model=None
	build_fn=<function create_model at 0x000001C0FC33F920>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=25
	n_hidden=11
	n_neurons=27
	learning_rate=0.00014593446517329207
	activation=relu
)


# DT

In [7]:
def loss_function(predt: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
        # Reshape the train data
        y = dtrain.get_label().reshape(predt.shape)
        # Calculate the error
        error = predt - y
        adjusted_demand = y.copy()
        # Loop over each datapoint and calculate the adjusted demand
        for i in range(error.shape[0]):
            line = error[i].copy()
            line[line < 0] = 0
            result = line * alpha_data
            adjusted_demand[i] = adjusted_demand[i] + result.sum(axis=1)
        # Return the gradient (difference between the prediction and the adjusted demand)
        return (predt - adjusted_demand).reshape(y.size)

NameError: name 'xgb' is not defined

In [20]:
x = np.array[1,2,3,4,5,6]
print(x.shape)

TypeError: 'builtin_function_or_method' object is not subscriptable

In [63]:
from typing import Dict, List, Tuple
import tensorflow as tf
import numpy as np
import xgboost as xgb

# Parameters for multi-item newsvendor problem
prices = np.array([0.3, 0.5, 0.6, 0.5, 0.5, 0.5])  # price data
costs = np.array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06])  # cost data
salvages = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01])  # salvage data
underage_data = prices - costs
overage_data = costs - salvages

alpha_data = np.array([  # alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
])

def make_nvps_loss_b(alpha, u, o):

    # transofrm the alpha, u, o to tensors
    u = tf.convert_to_tensor(underage_data, dtype=tf.float32) #underage costs
    o = tf.convert_to_tensor(overage_data, dtype=tf.float32) #overage costs
    alpha = tf.convert_to_tensor(alpha_data, dtype=tf.float32) #substitution matrix
    u = tf.reshape(u, (6,1))
    o = tf.reshape(o, (6,1))

    # define the loss function
    @tf.autograph.experimental.do_not_convert
    def nvps_loss(y_true, y_pred):
        q = tf.maximum(y_pred, 0.)

        demand_increase = 0
        adjusted_demand = 0
        profits = np.zeros(y_true.shape[0])
        profits_a = np.zeros(y_true.shape[0])
        profits_b = np.zeros(y_true.shape[0])
        # Calculate the demand increase for each product due to substitutions from other products
        for i in range(y_true.shape[0]):
            pred = tf.maximum(tf.reshape(y_pred[i], (6,1)),0.)
            true = tf.reshape(y_true[i], (6,1))

            print(pred.shape, true.shape, alpha.shape)
            demand_increase = tf.matmul(tf.transpose(tf.maximum(0.0, true - pred)), alpha)
            # Adjusted demand is the original demand plus the increase due to substitutions
            print("check1")
            adjusted_demand = y_true[i] + demand_increase
            print("check2")
            print(pred.shape, u.shape, o.shape, adjusted_demand.shape)
            profits_a[i] = tf.matmul(pred,tf.transpose(u)) 
            print("check3")
            profits_b[i] = tf.matmul(tf.maximum(0.0,pred - adjusted_demand), tf.transpose(u+o))
            print("check4")
            profits[i] = profits_a[i] - profits_b[i]
            print("check3")

        return -tf.math.reduce_mean(profits)
    
    return nvps_loss

def custom_rmse_model():
    loss_function = make_nvps_loss_b(alpha_data, underage_data, overage_data)

    def compute_objectives(predt, dtrain):
        y_pred = tf.convert_to_tensor(predt, dtype=tf.float32)
        y = dtrain.get_label().reshape(predt.shape)
        y_true = tf.convert_to_tensor(y, dtype=tf.float32)

        with tf.GradientTape() as tape_hessian:
            with tf.GradientTape() as tape_gradient:
                tape_gradient.watch(y_pred)
                tape_hessian.watch(y_pred)
                loss = loss_function(y_true, y_pred)
            grads = tape_gradient.gradient(loss, y_pred)
        hessians = tape_hessian.gradient(grads, y_pred)
        return grads.numpy(), hessians.numpy()

    def rmse(predt, dtrain):
        y = dtrain.get_label()
        v = np.sqrt(np.mean(np.square(y - predt)))
        return "PyRMSE", v

    X, y = X_train, target_train  # Assuming X_train, target_train are defined elsewhere
    Xy = xgb.DMatrix(X, label=y)
    results = {}
    booster = xgb.train(
        {
            "tree_method": "hist",
            "objective": "reg:squarederror"
        },
        dtrain=Xy,
        num_boost_round=128,
        obj=compute_objectives,
        evals=[(Xy, "Train")],
        evals_result=results,
        feval=rmse
    )
    y_pred = booster.predict(Xy)
    print("Predictions:", y_pred)

custom_rmse_model()


(6, 1) (6, 1) (6, 6)
check1
check2
(6, 1) (6, 1) (6, 1) (1, 6)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


ValueError: setting an array element with a sequence.

In [12]:
from typing import Dict, List, Tuple
import tensorflow as tf
import numpy as np
import xgboost as xgb

# Parameters for multi-item newsvendor problem
prices = np.array([0.3, 0.5, 0.6, 0.5, 0.5, 0.5]) #price data
costs = np.array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06]) #cost data
salvages = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01]) #salvage data
underage_data = prices - costs 
overage_data = costs - salvages 


alpha_data = np.array([             #alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
])

def make_nvps_loss(alpha, u, o):

    # transofrm the alpha, u, o to tensors
    u = tf.convert_to_tensor(underage_data, dtype=tf.float32) #underage costs
    o = tf.convert_to_tensor(overage_data, dtype=tf.float32) #overage costs
    alpha = tf.convert_to_tensor(alpha_data, dtype=tf.float32) #substitution matrix

    # define the loss function
    @tf.autograph.experimental.do_not_convert
    def nvps_loss(y_true, y_pred):
        q = tf.maximum(y_pred, 0.)

        print('ypred', y_pred)
        print('ytrue', y_true)

        # Calculate the demand increase for each product due to substitutions from other products
        demand_increase = tf.matmul( tf.maximum(0.0, y_true - y_pred),alpha)
        # Adjusted demand is the original demand plus the increase due to substitutions
        adjusted_demand = y_true + demand_increase

        profits = tf.matmul(q,tf.transpose(u)) - tf.matmul(tf.maximum(0.0,q - adjusted_demand), tf.transpose(u+o))

        return -tf.math.reduce_mean(profits)
    
    return nvps_loss

def custom_rmse_model() -> None:

    loss_function = make_nvps_loss(alpha_data, underage_data, overage_data)
       
    def compute_objectives(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[np.ndarray, np.ndarray]:
            # y_true and y_pred need to be TensorFlow tensors

        y_pred = tf.convert_to_tensor(predt, dtype=tf.float32)
        y_true = tf.convert_to_tensor(dtrain.get_label().reshape(predt.shape), dtype=tf.float32)
        with tf.GradientTape() as tape_hessian:
            with tf.GradientTape() as tape_gradient:
                tape_gradient.watch(y_pred)
                tape_hessian.watch(y_pred)
                loss = loss_function(y_true, y_pred)
            
            grads = tape_gradient.gradient(loss, y_pred)
    
        hessians = tape_hessian.gradient(grads, y_pred)
        return grads, hessians

    def rmse(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
        y = dtrain.get_label().reshape(predt.shape)
        v = np.sqrt(np.sum(np.power(y - predt, 2)))
        return "PyRMSE", v

    X, y = X_train, target_train
    print(X.shape, y.shape)

    Xy = xgb.DMatrix(X, y)
    results: Dict[str, Dict[str, List[float]]] = {}
    booster = xgb.train(
        {
            "tree_method": "hist",
            "num_target": y.shape[1],
            "multi_strategy": "multi_output_tree",
        },
        dtrain=Xy,
        num_boost_round=128,
        obj=compute_objectives,
        evals=[(Xy, "Train")],
        evals_result=results,
        custom_metric=rmse,
    )

    y_predt = booster.inplace_predict(X)
    print(y_predt)

model_XGB = custom_rmse_model()


(800, 15) (800, 6)
ypred tf.Tensor(
[[0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5]
 ...
 [0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5]], shape=(800, 6), dtype=float32)
ytrue tf.Tensor(
[[59. 73. 63. 35. 81. 84.]
 [49. 60. 52. 29. 67. 69.]
 [59. 73. 63. 35. 81. 84.]
 ...
 [55. 69. 59. 31. 77. 79.]
 [49. 61. 52. 28. 68. 70.]
 [45. 55. 48. 28. 61. 63.]], shape=(800, 6), dtype=float32)


InvalidArgumentError: {{function_node __wrapped____MklMatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} In[1] ndims must be >= 2 [Op:MatMul] name: 

In [ ]:
import numpy as np
import xgboost as xgb

def gradient(predt, dtrain):
    y_true = dtrain.get_label()
    y_pred = predt

    # Calculate the demand increase for each product due to substitutions from other products
    demand_increase = np.dot(np.maximum(0.0, y_true - y_pred), alpha_data)
    # Adjusted demand is the original demand plus the increase due to substitutions
    adjusted_demand = y_true + demand_increase

    q = np.maximum(y_pred, 0.)
    profits = np.dot(q, underage_data.T) - np.dot(np.maximum(0.0, q - adjusted_demand), (underage_data + overage_data).T)

    # Calculate gradient
    grad = -np.mean(profits)

    return grad.reshape(y_true.size)

def hessian(predt, dtrain):
    y_true = dtrain.get_label()
    # Return a matrix of ones with the same shape as y_true
    return np.ones_like(y_true).reshape(y_true.size)

def nvps_obj(predt, dtrain):
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

# Create a DMatrix from your training data
dtrain = xgb.DMatrix(X_train, label=target_train)

# Define the parameters for the xgb.train function
params = {
    'tree_method': 'hist',
    'num_target': 6,
    'multi_strategy': 'multi_output_tree',
    'n_boost_round': 128,
    'learning_rate': 0.1,
    'random_state': 42
}

# Train the model
model = xgb.train(params, dtrain, obj=nvps_obj)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:35:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


ValueError: operands could not be broadcast together with shapes (4800,) (800,6) 

In [ ]:
import numpy as np


def nvps_obj(y_pred, y_true):
    # Convert inputs to numpy arrays
    print('true: ' + str(np.shape(y_true)))
    print('pred: ' + str(np.shape(y_pred)))
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate the demand increase for each product due to substitutions from other products
    demand_increase = np.dot(np.maximum(0.0, y_true - y_pred), alpha_data)
    # Adjusted demand is the original demand plus the increase due to substitutions
    adjusted_demand = y_true + demand_increase

    q = np.maximum(y_pred, 0.)
    profits = np.dot(q, underage_data.T) - np.dot(np.maximum(0.0, q - adjusted_demand), (underage_data + overage_data).T)

    # Calculate gradient and hessian
    grad = -np.mean(profits)
    hess = np.ones_like(y_true)

    return grad, hess


from xgboost import XGBRegressor

model = XGBRegressor(objective=nvps_obj, n_estimators=100, learning_rate=0.1, random_state=42, 
                         num_target = 6, tree_method='hist', multi_strategy="multi_output_tree")

model.fit(X_train, target_train)

true: (800, 6)
pred: (4800,)


ValueError: operands could not be broadcast together with shapes (800,6) (4800,) 

In [ ]:
print(np.shape(target_train))

(800, 6)


true: (800, 6)
pred: (4800,)


ValueError: operands could not be broadcast together with shapes (800,6) (4800,) 

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Specify the parameter grid for RandomizedSearchCV
param_distribs = {
    'estimator__boosting_type': ['gbdt', 'dart', 'goss'],
    'estimator__num_leaves': [10, 20, 30, 40, 50],
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'estimator__feature_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_freq': [3, 4, 5, 6, 7],
}

critical_ratio = np.mean(underage_data) / (np.mean(overage_data) + np.mean(underage_data))
cr = critical_ratio

# Create the LightGBM model
model = lgb.LGBMRegressor(objective='quantile', alpha=cr, metric='quantile', verbose=0)

# Wrap the model with MultiOutputRegressor
model = MultiOutputRegressor(model)

# Perform RandomizedSearchCV
rnd_search = RandomizedSearchCV(model, param_distributions=param_distribs, n_iter=10, cv=3)

# Fit the model
rnd_search.fit(X_train, target_train)
print(rnd_search.best_params_)



[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\la

In [ ]:
# Get the best model
best_model = rnd_search.best_estimator_

# Predict the demand using the best model
target_pred_DT = best_model.predict(X_test)
target_pred_DT = pd.DataFrame(np.row_stack(target_pred_DT))
print(target_pred_DT)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 

# Costs

In [ ]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0

for i in range(len(target_test)):
    for j in range(len(target_test.columns)):
        # Calculate understock and overstock costs
        cost_ANN = 0
        cost_DT = 0

        if target_pred_ANN.iloc[i, j] < target_test.iloc[i, j]:
            cost_ANN = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_ANN.iloc[i, j]))

        if target_pred_ANN.iloc[i, j] > target_test.iloc[i, j]:
            cost_ANN = (costs[j] - salvages[j]) * (np.round(target_pred_ANN.iloc[i, j]) - target_test.iloc[i, j])
        
        if target_pred_DT.iloc[i, j] < target_test.iloc[i, j]:
            cost_DT = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_DT.iloc[i, j]))

        if target_pred_DT.iloc[i, j] > target_test.iloc[i, j]:
            cost_DT = (costs[j] - salvages[j]) * (np.round(target_pred_DT.iloc[i, j]) - target_test.iloc[i, j])
        
        # Calculate the total costs for the week
        overall_costs_ANN += cost_ANN
        overall_costs_DT += cost_DT

# Print the overall costs
print('Overall costs for ANN: ', int(overall_costs_ANN))
print('Overall costs for DT: ', int(overall_costs_DT))

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'